&emsp;&emsp;接下来我们就借助ModelScope Notebook来完成llama3大模型部署调用入门实验。

- huggingface Llama3模型主页：https://huggingface.co/meta-llama/     
- Github主页：https://github.com/meta-llama/llama3/tree/main
- ModelScope Llama3-8b模型主页：https://www.modelscope.cn/models/LLM-Research/Meta-Llama-3-8B-Instruct/summary

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404171645681.png" alt="开源大模型下载方法" style="zoom:33%;" />

### 2.本地项目文件下载与transformer库运行

- 借助modelscope进行模型下载

In [1]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

2024-04-19 15:31:49,493 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-04-19 15:31:49,496 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-04-19 15:31:49,496 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-04-19 15:31:49,497 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-04-19 15:31:49,856 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 55e7043102d017111a56be6e6d7a6a16 and a total number of 972 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('LLM-Research/Meta-Llama-3-8B-Instruct')

Downloading: 100%|██████████| 654/654 [00:00<00:00, 5.15MB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 428kB/s]
Downloading: 100%|██████████| 126/126 [00:00<00:00, 927kB/s]
Downloading: 100%|██████████| 7.62k/7.62k [00:00<00:00, 10.5MB/s]
Downloading: 100%|█████████▉| 4.63G/4.63G [00:13<00:00, 379MB/s]
Downloading: 100%|█████████▉| 4.66G/4.66G [00:13<00:00, 374MB/s]
Downloading: 100%|█████████▉| 4.58G/4.58G [00:13<00:00, 357MB/s]
Downloading: 100%|█████████▉| 1.09G/1.09G [00:03<00:00, 339MB/s]
Downloading: 100%|██████████| 23.4k/23.4k [00:00<00:00, 61.1MB/s]
Downloading: 100%|██████████| 36.3k/36.3k [00:00<00:00, 18.6MB/s]
Downloading: 100%|██████████| 73.0/73.0 [00:00<00:00, 600kB/s]
Downloading: 100%|██████████| 8.66M/8.66M [00:00<00:00, 65.8MB/s]
Downloading: 100%|██████████| 49.7k/49.7k [00:00<00:00, 11.4MB/s]
Downloading: 100%|██████████| 4.59k/4.59k [00:00<00:00, 8.31MB/s]


In [3]:
model_dir

'/mnt/workspace/.cache/modelscope/LLM-Research/Meta-Llama-3-8B-Instruct'

- 使用transformers库运行本地大模型

In [4]:
# AutoModelForCausalLM 是用于加载预训练的因果语言模型（如GPT系列）
# 而 AutoTokenizer 是用于加载与这些模型匹配的分词器。
from transformers import AutoModelForCausalLM, AutoTokenizer

# 这行设置将模型加载到 GPU 设备上，以利用 GPU 的计算能力进行快速处理
device = "cuda" 

# 加载了一个因果语言模型。
# model_dir 是模型文件所在的目录。
# torch_dtype="auto" 自动选择最优的数据类型以平衡性能和精度。
# device_map="auto" 自动将模型的不同部分映射到可用的设备上。
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)

# 加载与模型相匹配的分词器。分词器用于将文本转换成模型能够理解和处理的格式。
tokenizer = AutoTokenizer.from_pretrained(model_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.97s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 加载与模型相匹配的分词器。分词器用于将文本转换成模型能够理解和处理的格式
prompt = "你好，请介绍下你自己。"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 使用分词器的 apply_chat_template 方法将上面定义的消息列表转换为一个格式化的字符串，适合输入到模型中。
# tokenize=False 表示此时不进行令牌化，add_generation_prompt=True 添加生成提示。
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 将处理后的文本令牌化并转换为模型输入张量，然后将这些张量移至之前定义的设备（GPU）上。
model_inputs = tokenizer([text], return_tensors="pt").to(device)

In [6]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [7]:
print(response)

😊 Ni Hao! I'm a helpful assistant, designed to assist and communicate with users in a friendly and efficient manner. I'm a large language model, trained on a massive dataset of text from various sources, which enables me to understand and respond to a wide range of questions and topics.

I can help with various tasks, such as:

* Answering questions on various subjects, including science, history, technology, and more
* Providing definitions and explanations for complex terms and concepts
* Generating text, such as articles, stories, and even entire books
* Translating text from one language to another
* Summarizing long pieces of text into shorter, more digestible versions
* Offering suggestions and ideas for creative projects
* And much more!

I'm constantly learning and improving, so please bear with me if I make any mistakes. I'm here to help and provide assistance to the best of my abilities. What can I help you with today? 🤔assistant

😊assistant

I see you responded with a smile!

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404191538918.png" alt="image-20240419153836830" style="zoom:33%;" />

- 使用ollama进行调用

&emsp;&emsp;当然，除了可以使用上述方法进行开源大模型部署调用外，我们也可以使用一些大模型部署和调用工具，来快速完成各类大模型部署。目前来看，最常用的开源大模型部署和调用工具有两类，其一是ollama、其二是vLLM。这两款工具定位类似，但功能实现各有侧重。ollama更加侧重于为个人用户提供更加便捷的开源模型部署和调用服务，ollama提供了openai风格的调用方法、GPU和CPU混合运行模式、以及更加便捷的显存管理方法，而vLLM则更加适用于企业级应用场景，采用的是服务端和客户端分离的模式，更适合企业级项目使用。

&emsp;&emsp;这里我们以ollama为例，介绍借助工具部署调用开源大模型方法。ollama部署和调用开源大模型方式非常简单，首先打开服务器命令行页面并运行安装脚本：

```bash
curl -fsSL https://ollama.com/install.sh | sh
```

稍等片刻即可完成安装：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181657309.png" alt="image-20240418165711192" style="zoom:33%;" />

然后开启ollama服务：

```bash
ollama serve
```

然后即可使用如下命令安装和在命令行中调用llama3大模型：

```bash
ollama run llama3
```

然后回到代码环境中：

In [1]:
!pip install openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 680.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 687.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 716.2 kB/s eta 0:00:00a 0:00:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',  # required but ignored
)
chat_completion = client.chat.completions.create(
    messages=[
        {'role': 'user','content': '你好，请介绍下你自己',}
    ],
    model='llama3',
)
#打印返回内容
chat_completion.choices[0]
print(chat_completion.choices[0].message.content)

In [10]:
def run_chat_session():
    # 初始化客户端
    client = OpenAI(
        base_url='http://localhost:11434/v1/',
        api_key='ollama',  # API key is required but ignored for local model
    )
    
    # 初始化对话历史
    chat_history = []
    
    # 启动对话循环
    while True:
        # 获取用户输入
        user_input = input("你: ")
        
        # 检查是否退出对话
        if user_input.lower() == 'exit':
            print("退出对话。")
            break
        
        # 更新对话历史
        chat_history.append({'role': 'user', 'content': user_input})
        
        # 调用模型获取回答
        try:
            chat_completion = client.chat.completions.create(
                messages=chat_history,
                model='llama3',
            )
            # 获取最新回答，适当修改以适应对象属性
            model_response = chat_completion.choices[0].message.content
            print("AI: ", model_response)
            
            # 更新对话历史
            chat_history.append({'role': 'assistant', 'content': model_response})
        except Exception as e:
            print("发生错误:", e)
            break

In [12]:
run_chat_session()

你:  你好，请介绍下你自己


AI:  😊 Ni Hao (Hello)! I'm LLaMA, a large language model trained by a team of researcher at Meta AI. My primary function is to generate human-like text responses to user input, which can range from simple queries to more complex topics.

I was created using a combination of natural language processing (NLP) and machine learning techniques, including transformer architectures and masked language modeling. This allows me to understand and respond to user input in a way that simulates conversation.

Some of my key features include:

* **Conversational capabilities**: I can engage in back-and-forth conversations with users, responding to their questions and statements in a natural-sounding manner.
* **Language understanding**: I can comprehend complex queries, nuances of language, and even idioms to provide accurate responses.
* **Creativity**: I have been trained on vast amounts of text data, which enables me to generate creative content, such as stories, dialogues, or even entire scripts

你:  好的，请问什么是机器学习？


AI:  😊 Machine learning (ML) is a subfield of artificial intelligence (AI) that involves training algorithms to make predictions or take actions based on data, without being explicitly programmed.

In traditional programming, developers write explicit rules and instructions for the computer to follow. In machine learning, the algorithm learns from the data provided and adjusts its behavior accordingly. This allows it to improve its performance over time, as long as the data is relevant and sufficient.

Machine learning has many applications, including:

1. **Image recognition**: Computers can learn to identify objects, people, and animals in images.
2. **Speech recognition**: Algorithms can recognize spoken words and phrases.
3. **Natural language processing** (NLP): Machines can understand and generate human-like text.
4. **Predictive modeling**: ML algorithms can forecast future trends or outcomes based on past data.
5. **Game playing**: AI systems can learn to play games like chess,

你:  exit


退出对话。


## 二、Llama 3高效微调流程

&emsp;&emsp;在完成了Llama3模型的快速部署之后，接下来我们尝试围绕Llama3的中文能力进行微调。

&emsp;&emsp;所谓微调，通俗理解就是围绕大模型进行参数修改，从而永久性的改变模型的某些性能。而大模型微调又分为全量微调和高效微调两种，所谓全量微调，指的是调整大模型的全部参数，而高效微调，则指的是调整大模型的部分参数，目前常用的高效微调方法包括LoRA、QLoRA、p-Tunning、Prefix-tunning等。而只要大模型的参数发生变化，大模型本身的性能和“知识储备”就会发生永久性改变。在通用大模型往往只具备通识知识的当下，为了更好的满足各类不同的大模型开发应用场景，大模型微调已几乎称为大模型开发人员的必备基础技能。

- LLaMA-Factory项目介绍

&emsp;&emsp;LLaMA Factory是一个在GitHub上开源的项目，该项目给自身的定位是：提供一个易于使用的大语言模型（LLM）微调框架，支持LLaMA、Baichuan、Qwen、ChatGLM等架构的大模型。更细致的看，该项目提供了从预训练、指令微调到RLHF阶段的开源微调解决方案。截止目前（2024年3月1日）支持约120+种不同的模型和内置了60+的数据集，同时封装出了非常高效和易用的开发者使用方法。而其中最让人喜欢的是其开发的LLaMA Board，这是一个零代码、可视化的一站式网页微调界面，它允许我们通过Web UI轻松设置各种微调过程中的超参数，且整个训练过程的实时进度都会在Web UI中进行同步更新。

&emsp;&emsp;简单理解，通过该项目我们只需下载相应的模型，并根据项目要求准备符合标准的微调数据集，即可快速开始微调过程，而这样的操作可以有效地将特定领域的知识注入到通用模型中，增强模型对特定知识领域的理解和认知能力，以达到“通用模型到垂直模型的快速转变”。

#### 1. LLaMA-Factory私有化部署

- **Step 1. 下载LLaMA-Factory的项目文件**

&emsp;&emsp;进入LLaMA-Factory的官方Github，地址：https://github.com/hiyouga/LLaMA-Factory ， 在 GitHub 上将项目文件下载到有两种方式：克隆 (Clone) 和 下载 ZIP 压缩包。推荐使用克隆 (Clone)的方式。我们首先在GitHub上找到其仓库的URL。

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240227222232866.png" width=80%></div>

&emsp;&emsp;在执行命令之前，需要先安装git软件包，执行命令如下：
```bash
apt install git
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181739271.png" alt="image-20240418173924246" style="zoom:33%;" />

然后再主目录中下载项目文件：

```bash
cd
git clone https://github.com/hiyouga/LLaMA-Factory.git
```

下载完成后即可看到LLaMA-Factory目录：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181742391.png" alt="image-20240418174251349" style="zoom:33%;" />

- **Step 2. 升级pip版本**

&emsp;&emsp;建议在执行项目的依赖安装之前升级 pip 的版本，如果使用的是旧版本的 pip，可能无法安装一些最新的包，或者可能无法正确解析依赖关系。升级 pip 很简单，只需要运行命令如下命令：

```bash
python -m pip install --upgrade pip
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181739896.png" alt="image-20240418173954874" style="zoom:40%;" />

- **Step 3. 使用pip安装LLaMA-Factory项目代码运行的项目依赖**

&emsp;&emsp;在LLaMA-Factory中提供的 `requirements.txt`文件包含了项目运行所必需的所有 Python 包及其精确版本号。使用pip一次性安装所有必需的依赖，执行命令如下：

```bash
pip install -r requirements.txt --index-url https://mirrors.huaweicloud.com/repository/pypi/simple
```

通过上述步骤就已经完成了LLaMA-Factory模型的完整私有化部署过程。

#### 3.基于LLaMA-Factory的Llama3中文能力微调过程

&emsp;&emsp;基于LLaMA-Factory的完整高效微调流程如下，本次实验中我们将借助Llama-Factory的alpaca_data_zh_51k数据集进行微调，暂不涉及关于数据集上传和修改数据字典事项：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181757297.png" alt="image-20240418175747249" style="zoom:33%;" />

微调流程如下：

- **Step 1. 查看微调中文数据集数据字典**

&emsp;&emsp;我们找到`./LLaMA-Factory`目录下的data文件夹：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404191954405.png" alt="image-20240419195409366" style="zoom:33%;" />

查看dataset_info.json:

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404191951423.png" alt="image-20240419195134391" style="zoom:33%;" />

找到当前数据集名称：alpaca_zh。数据集情况如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404191955498.png" alt="image-20240419195555431" style="zoom:30%;" />

- **Step 3. 创建微调脚本**

&emsp;&emsp;所谓高效微调框架，我们可以将其理解为很多功能都进行了高层封装的工具库，为了使用这些工具完成大模型微调，我们需要编写一些脚本（也就是操作系统可以执行的命令集），来调用这些工具完成大模型微调。这里我们需要先回到LlaMa-Factory项目主目录下：

```bash
cd ..
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181813703.png" style="zoom:50%;" />

然后创建一个名为`single_lora_llama3.sh`的脚本（脚本的名字可以自由命名）。这里我们可以使用使用vim创建这个脚本文件，同时也可以直接把课件中的single_lora_qwen.sh文件直接上传到jupyter主目录下，然后再用cp命令复制到LlaMa-Factory主目录下。这里我们先简单查看这个脚本文件内容：

```bash
#!/bin/bash
export CUDA_DEVICE_MAX_CONNECTIONS=1

export NCCL_P2P_DISABLE="1"
export NCCL_IB_DISABLE="1"


# 如果是预训练，添加参数       --stage pt \
# 如果是指令监督微调，添加参数  --stage sft \
# 如果是奖励模型训练，添加参数  --stage rm \
# 添加 --quantization_bit 4 就是4bit量化的QLoRA微调，不添加此参数就是LoRA微调 \



CUDA_VISIBLE_DEVICES=0 python src/train_bash.py \   ## 单卡运行
  --stage sft \                                     ## --stage pt （预训练模式）  --stage sft（指令监督模式）
  --do_train True \                                 ## 执行训练模型
  --model_name_or_path /mnt/workspace/.cache/modelscope/LLM-Research/Meta-Llama-3-8B-Instruct \     ## 模型的存储路径
  --dataset alpaca_zh \                                ## 训练数据的存储路径，存放在 LLaMA-Factory/data路径下
  --template llama3 \                                 ## 选择Qwen模版
  --lora_target q_proj,v_proj \                     ## 默认模块应作为
  --output_dir /mnt/workspace/.cache/modelscope/single_lora_llama3_checkpoint \        ## 微调后的模型保存路径
  --overwrite_cache \                               ## 是否忽略并覆盖已存在的缓存数据
  --per_device_train_batch_size 2 \                 ## 用于训练的批处理大小。可根据 GPU 显存大小自行设置。
  --gradient_accumulation_steps 64 \                 ##  梯度累加次数
  --lr_scheduler_type cosine \                      ## 指定学习率调度器的类型
  --logging_steps 5 \                               ## 指定了每隔多少训练步骤记录一次日志。这包括损失、学习率以及其他重要的训练指标，有助于监控训练过程。
  --save_steps 100 \                                ## 每隔多少训练步骤保存一次模型。这是模型保存和检查点创建的频率，允许你在训练过程中定期保存模型的状态
  --learning_rate 5e-5 \                            ## 学习率
  --num_train_epochs 1.0 \                          ## 指定了训练过程将遍历整个数据集的次数。一个epoch表示模型已经看过一次所有的训练数据。
  --finetuning_type lora \                          ## 参数指定了微调的类型，lora代表使用LoRA（Low-Rank Adaptation）技术进行微调。
  --fp16 \                                          ## 开启半精度浮点数训练
  --lora_rank 4 \                                   ## 在使用LoRA微调时设置LoRA适应层的秩。
```

> 注：实际脚本文件最好不要出现中文备注，否则容易出现编辑格式导致的问题。

当我们拿到这个脚本文件后，首先将其上传到ModelScope NoteBook主目录下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181821260.png" alt="image-20240418182117239" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181822353.png" alt="image-20240418182250323" style="zoom:33%;" />

然后使用cp命令回到当前项目主目录下，查看脚本情况：

```bash
cd /mnt/workspace
ll
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181824417.png" alt="image-20240418182447380" style="zoom:33%;" />

然后将其复制到LlaMa-Factory主目录下，并简单查看脚本位置：

```bash
cp single_lora_llama3.sh ~/LLaMA-Factory
cd ~/LLaMA-Factory/
ll
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181827931.png" alt="image-20240418182730878" style="zoom:33%;" />

然后为了保险起见，我们需要对齐格式内容进行调整，以满足Ubuntu操作系统运行需要（此前是从Windows系统上复制过去的文件，一般都需要进行如此操作）：

```bash
sed -i 's/\r$//' ./single_lora_llama3.sh
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181829268.png" alt="image-20240418182941248" style="zoom:33%;" />

- **Step 4. 运行微调脚本，获取模型微调权重**

&emsp;&emsp;当我们准备好微调脚本之后，接下来即可围绕当前模型进行微调了。这里我们直接在命令行中执行sh文件即可，注意运行前需要为该文件增加权限：

```bash
chmod +x ./single_lora_llama3.sh
./single_lora_llama3.sh
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404191958475.png" alt="image-20240419195843359" style="zoom:33%;" />

当微调结束之后，我们就可以在当前主目录下看到新的模型权重文件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181837915.png" alt="image-20240418183720857" style="zoom:33%;" />

- **Step 5. 合并模型权重，获得微调模型**

&emsp;&emsp;接下来我们需要将该模型权重文件和此前的原始模型权重文件进行合并，才能获得最终的微调模型。LlaMa-Factory中已经为我们提供了非常完整的模型合并方法，同样，我们只需要编写脚本文件来执行合并操作即可，即`llama3_merge_model.sh`。同样，该脚本文件也可以按照此前single_lora_llama3.sh脚本相类似的操作，就是将课件中提供的脚本直接上传到Jupyter主目录下，再复制到LlaMa-Factory主目录下进行运行。

&emsp;&emsp;首先简单查看llama3_merge_model.sh脚本文件内容：

```bash
#!/bin/bash

python src/export_model.py \               ## 用于执行合并功能的Python代码文件
  --model_name_or_path /mnt/workspace/.cache/modelscope/LLM-Research/Meta-Llama-3-8B-Instruct \  ## 原始模型文件
  --adapter_name_or_path /mnt/workspace/.cache/modelscope/single_lora_llama3_checkpoint                ## 微调模型权重文件
  --template llama3 \                        ## 模型模板名称
  --finetuning_type lora \                 ## 微调框架名称
  --export_dir  /mnt/workspace/.cache/modelscope/llama3_lora \                          ## 合并后新模型文件位置
  --export_size 2 \
  --export_legacy_format false
```

> 注：实际脚本文件最好不要出现中文备注，否则容易出现编辑格式导致的问题。

同样，我们将课件中的merge_model.sh文件上传到在线Jupyter Notebook中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181844623.png" alt="image-20240418184454597" style="zoom:33%;" />

然后使用cp命令将其复制到LlaMa-Fcotry项目主目录下：

```bash
cd /mnt/workspace
cp llama3_merge_model.sh ~/LLaMA-Factory
cd ~/LLaMA-Factory/
chmod +x ./llama3_merge_model.sh
sed -i 's/\r$//' ./llama3_merge_model.sh
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181847203.png" alt="image-20240418184759149" style="zoom:33%;" />

然后运行脚本，进行模型合并：

```bash
./llama3_merge_model.sh
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181925276.png" alt="image-20240418192517123" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181925786.png" alt="image-20240418192548707" style="zoom:33%;" />

接下来即可查看刚刚获得的新的微调模型：

```bash
cd /mnt/workspace/.cache/modelscope
ll
```

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181927237.png" alt="image-20240418192728206" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202404181930515.png" alt="image-20240418193024475" style="zoom:33%;" />

- **Step 6. 测试微调效果**

In [16]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

2024-04-19 21:11:48,141 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-04-19 21:11:48,144 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-04-19 21:11:48,145 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-04-19 21:11:48,488 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 55e7043102d017111a56be6e6d7a6a16 and a total number of 972 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
model_dir = '/mnt/workspace/.cache/modelscope/llama3_lora1'

In [18]:
model_dir

'/mnt/workspace/.cache/modelscope/llama3_lora1'

In [19]:
# AutoModelForCausalLM 是用于加载预训练的因果语言模型（如GPT系列）
# 而 AutoTokenizer 是用于加载与这些模型匹配的分词器。
from transformers import AutoModelForCausalLM, AutoTokenizer

# 这行设置将模型加载到 GPU 设备上，以利用 GPU 的计算能力进行快速处理
device = "cuda" 

# 加载了一个因果语言模型。
# model_dir 是模型文件所在的目录。
# torch_dtype="auto" 自动选择最优的数据类型以平衡性能和精度。
# device_map="auto" 自动将模型的不同部分映射到可用的设备上。
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)

# 加载与模型相匹配的分词器。分词器用于将文本转换成模型能够理解和处理的格式。
tokenizer = AutoTokenizer.from_pretrained(model_dir)

Loading checkpoint shards: 100%|██████████| 17/17 [01:03<00:00,  3.76s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# 加载与模型相匹配的分词器。分词器用于将文本转换成模型能够理解和处理的格式
prompt = "你好，请介绍下你自己。"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 使用分词器的 apply_chat_template 方法将上面定义的消息列表转换为一个格式化的字符串，适合输入到模型中。
# tokenize=False 表示此时不进行令牌化，add_generation_prompt=True 添加生成提示。
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 将处理后的文本令牌化并转换为模型输入张量，然后将这些张量移至之前定义的设备（GPU）上。
model_inputs = tokenizer([text], return_tensors="pt").to(device)

In [21]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
response

'嗨！我是一个人工智能助手，旨在帮助您回答问题、完成任务和提供有用的信息。'